# Auswerten eines Schach-Endspiels

## Imports, die für dieses Notebook benötigt werden

In [ ]:
import chess
import stockfish
from IPython.display import clear_output
from IPython.display import display
import json
from stockfish import Stockfish
import time

## Konfiguration und globale Variablen

An dieser Stelle kann eine FEN eingegeben werden, welche die Figuren enthält, mit welcher die $S_n$ Sequenz aus der
ebenfalls einzugebenden Datei berechnet wurde.

Weiter muss der Pfad zu einer Stockfish-Installation angegeben werden.

In [ ]:
FEN = "4k3/8/8/8/8/8/8/R3K3 w - - 0 1"
S_N_FILE = "S_n_seq_12_01.json"
STOCKFISH_PATH = "./stockfish_14.1/stockfish_14.1_win_x64_avx2.exe"
VERBOSE = False

STOCKFISH = stockfish.Stockfish(STOCKFISH_PATH)

## Import der Daten

Die Daten wurden als FEN in der JSON-Datei Serialisiert.
Zum Initialisieren der Liste werden alle FENs gelesen und Board-Objekte erstellt.

In [ ]:
def load_s_n_sequence(filepath):
    s_n_sequence_new = []
    f = open(filepath, "r")
    tmp = json.loads(f.read())
    for item in tmp:
        tmp_list = []
        for board in item:
            tmp_list.append(chess.Board(board))
        s_n_sequence_new.append(tmp_list)
    f.close()
    return s_n_sequence_new

## die *find_board_in_sequence* Hilfsfunktion

Diese Funktion durchsucht eine $S_n$ Sequenz nach dem ersten Vorkommen eines übergebenen Board-Objekts.

Funktionsargumente:
* situation: Das Board (als Objekt), welches gefunden werden soll
* sequence: Die $S_n$ Sequenz, in welcher das Board gesucht wird

Ergebnis der Ausführung:
* Die Funktion hat zwei mögliche Rückgaben:
  * Ein Tupel mit $S_n$ Index (z.B. $S_3$) und Board-Index (z.B. 100).
    Dieses Tupel drückt aus, wo in der Sequenz das Board gefunden wurde.
  * Das Tupel (-1,-1). Dies drückt aus, dass das Board nicht gefunden wurde.

Nebeneffekte:
Die Funktion verändert keinen der übergebenen Parameter.

Algorithmus:
1. Über die $S_n$ Sequenz iterieren.
2. Über jedes Board in einem spezifischem $S_n$ iterieren.
3. Das Board mit dem _situation_ Objekt vergleichen.
    1. Wenn das Board übereinstimmt, die Indizes zurückgeben.
    2. Wenn das Board nicht übereinstimmt, weitersuchen.

In [ ]:
def find_board_in_sequence(situation, sequence):
    board_str = (situation.turn, situation.__str__())

    for i in range(len(sequence)):
        for j in range(len(sequence[i])):
            item = sequence[i][j]
            if board_str == (item.turn, item.__str__()):
                return i, j
    return -1,-1

## Die *find_next_move* Hilfsfunktion
Diese Funktion berechnet aus einem übergebenen Board den idealen Spielzug.
Aus Effizienzgründen gibt sie auch die Position des nächsten Boards in der $S_n$ Sequenz zurück.

Funktionsparameter:
* *curr_board*: Das Board, für welches der nächste Spielzug berechnet werden soll.
* *s_index*: Das $n$ eines $S_n$, in welchem sich *curr_board* befindet.
Wird aus effizienz Gründen übergeben.
* *s_n_sequence*: Die Liste mit allen $S_n$

Ergebnis der Ausführung:
* Rückgabe: Der nächsten Move oder der Wert -1 als Hinweis, dass kein Spielzug berechnet werden konnte

Nebeneffekte:
* Keine.

Algorithmus:
* Überprüfe, ob Weiß oder Schwarz am Zug ist.
* Wenn Weiß:
  * Führe alle möglichen moves durch, bis das Ergebnis-Board in $S_{n-1}$ gefunden wurde.
* Wenn Schwarz:
  * Nutze die *get_best_move* Funktion der Stockfish-Engine.
  * Berechne den s_index und board_index des Boards, nach dem Zug.
* Gebe den neuen s_index, board_index und den gefundenen move zurück.

In [ ]:
STOCKFISH = Stockfish(STOCKFISH_PATH)

def find_next_move(curr_board, s_index, s_n_sequence):
    STOCKFISH.set_fen_position(curr_board.fen())
    if curr_board.turn:
        if VERBOSE:
            print("---White:---")
            print("Starting in S" + str(s_index))
        for move in curr_board.legal_moves:
            curr_board.push(move)
            _tmp, board_index_tmp = find_board_in_sequence(curr_board, [s_n_sequence[s_index - 1]])
            s_index_tmp = s_index - 1
            if board_index_tmp != -1:
                if VERBOSE:
                    print("    Move: " + str(move))
                    print("    S" + str(s_index_tmp) + " - Board: " + str(board_index_tmp))
                    print("Ended in S" + str(s_index))
                curr_board.pop()
                return s_index_tmp, board_index_tmp, move
            curr_board.pop()

        return -1, -1, None
    else:
        if VERBOSE:
            print("---Black:---")
            print("Starting in S" + str(s_index))
        move = chess.Move.from_uci(STOCKFISH.get_best_move())
        curr_board.push(move)
        s_index, board_index = find_board_in_sequence(curr_board, s_n_sequence[:s_index])
        if VERBOSE:
            print("    Move: " + str(move))
            print("    S" + str(s_index) + " - Board: " + str(board_index))
            print("Ended in S" + str(s_index))
        curr_board.pop()
        return s_index, board_index, move

## Die *calculate_all_moves* Hilfsfunktion
Diese Funktion berechnet in einer Schleife alle Moves ausgehend von einer Fen, bis zu einem Gewinn.

Funktionsargumente:
* *fen*: Die FEN der Spielsituation, die berechnet werden soll.
* *s_n_sequence*: Die Liste der $S_n$, welche für die Figurenkonstellation berechnet wurde.

Ergebnis der Ausführung:
* Die Liste aller Moves, welche für das Board berechnet wurde oder der Wert None als Hinweis, dass kein Sieg erzielt
werden konnte.

Nebeneffekte:
* Keine.

Algorithmus:
* Einordnung der übergebenen FEN in der $S_n$ Sequenz finden.
* Solange $n > 0$ in einer Schleife den nächsten Spielzug berechnen.
* Die Liste der errechneten Spielzüge zurückgeben.

In [ ]:
def calculate_all_moves(fen, s_n_sequence):
    moves = []

    board = chess.Board(fen)

    s_index, board_index = find_board_in_sequence(board, s_n_sequence)

    while s_index > 0:
        curr_board = s_n_sequence[s_index][board_index].copy()
        s_index, board_index, next_move = find_next_move(curr_board, s_index, s_n_sequence)
        moves.append(next_move)

    if s_index == -1:
        return None

    return moves

## Die *stockfish_movelist* Hilfsfunktion
Diese Funktion sammelt alle Züge, welche Stockfish bis zum Sieg durchführt.

Funktionsparameter:
* *fen*: Die FEN des Boardes, für welches alle Züge berechnet werden sollen.

Ergebnis der Ausführung:
* Eine Liste an Spielzügen

Nebeneffekte:
* Keine.

Algorithmus:
* Solange das Spiel nicht beendet ist einen weiteren Zug berechnen
* Am Ende alle Züge zurückgeben.

In [ ]:
def stockfish_movelist(fen):
    moves = []

    board = chess.Board(fen)

    while not board.is_game_over():
        STOCKFISH.set_fen_position(board.fen())
        next_move = chess.Move.from_uci(STOCKFISH.get_best_move())
        board.push(next_move)
        moves.append(next_move)

    return moves

## Sie *show_movelist* Hilfsfunktion
Diese Funktion zeigt Spielzüge in einer Liste visuell auf dem Schachbrett.

Funktionsparameter:
* *fen*: Die FEN des Boardes, für welches alle Züge dargestellt werden sollen.
* *moves*: Die Liste der Spielzüge, welche dargestellt werden sollen.

Nebeneffekte:
* Keine.

In [ ]:
def show_movelist(fen, moves):
    presentation_board = chess.Board(fen)
    display(presentation_board)
    time.sleep(2)
    for move in moves:
        presentation_board.push(move)
        clear_output(wait=True)
        display(presentation_board)
        time.sleep(2)

## Berechnen der Züge, anzeigen dieser und ein Vergleich mit Stockfish

In [ ]:
S_N_Sequence = load_s_n_sequence(S_N_FILE)
Moves = calculate_all_moves(FEN, S_N_Sequence)
Stockfish_Moves = stockfish_movelist(FEN)

if Moves is not None:
    print("AI needed " + str(len(Moves)) + " moves to beat Stockfish as Black.")
else:
    print("AI found no way to beat Black.")

print("Stockfish needed " + str(len(Stockfish_Moves)) + " moves to win against itself.")

In [ ]:
show_movelist(FEN, Moves)